In [73]:
import pandas as pd
import lxml.html as lh
import requests
import time
import os
import igc_validation

In [82]:
def download_flight(flight_number):
    
    filename = "flights/flight_{}.igc".format(flight_number)
    headers = {'User-Agent': 'Mozilla/5.0'}

    payload = {'type':'igc',
               'flightID': str(flight_number),
               'file': '',
               'captchaStr': '',
               'captcha': '',
              }

    session = requests.Session()
    res = session.post('http://xc.rmhpa.org/download_igc.php', headers=headers, data=payload)

    with open(filename, 'wb') as outfile:
        outfile.write(res.content)
     
    
    logbook = []
    logbook.append({'igcfile': os.path.abspath(filename)})
    
    for flight in logbook:
        flight = igc_validation.parse_igc(flight)
    
    for flight in logbook:
        flight = igc_validation.crunch_flight(flight)
    
    for flight in logbook:
        flight['outputfilename'] = igc_validation.get_output_filename(flight['igcfile'])
        

This will take about 3 hours... I have about 200 of them.

In [85]:
df = pd.read_csv("clean/rmhpa_flights.csv")
records = df.to_dict(orient='records')
tic = time.time()

for i, rec in enumerate(records):
    url = rec['links']
    flight_number = url.split('/')[-1]
    
    #if not os.path.exists("flights/flight_{}.igc".format(flight_number)):    
    download_flight(flight_number)
    print(" #{} {} {}    {}% done....   {} seconds".format(flight_number, str(rec['index']), rec['distance'], round(i/len(records)*100), round(time.time()-tic)))
    print("---------------------------------------------")
    time.sleep(1)
        
    if i == 50:
        break

Flight date: 2013-06-18
Header (not implemented): OPLTPILOT: Benzie
Header (not implemented): OGTYGLIDERTYPE: 3
Header (not implemented): OGIDGLIDERID: None
Header (not implemented): ODTM100GPSDATUM: WGS-84
Header (not implemented): OCIDCOMPETITIONID:
Header (not implemented): OCCLCOMPETITION CLASS: None
Header (not implemented): OSITSite: None
Record Type L not implemented: XGD GpsDump version 4.74
Record Type L not implemented: XGD Downloaded 2013-06-22  00:54:54
Record Type G not implemented: EC2B349363E8B59F
 #732 1284 0.2 km    0% done....   0 seconds
---------------------------------------------
